In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/4040')

## 1. Load Packages

In [3]:

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
# !pip install imgaug
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,RMSprop, SGD
from utils import load_data, data_augmentation, show_sample, get_labels
from models import init_model_1, init_model_2, training, save_model

In [11]:
from models import init_model_1, init_model_2,resblock, training, save_model

# 2. Load Data Set & Data Path


In [5]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -qq 'tiny-imagenet-200.zip'

--2020-12-21 01:48:44--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  11.6MB/s    in 13s     

2020-12-21 01:48:57 (18.1 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [7]:
train_generator, val_generator, test_generator = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=None, seed=101, load_test=True)

Start downloading data...
Download complete.
Begin extracting...
Unzip complete.
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
Found 10000 validated image filenames belonging to 200 classes.
Testing data shape: (10000, 64, 64, 3)
End loading!


In [6]:
data_path = '/content/tiny-imagenet-200'
checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/4040/checkpoints'
model_path = '/content/drive/MyDrive/Colab Notebooks/4040/savedmodel'

# 3. Data augmentation

In [9]:
# parameters for data_augmentation function
# use ?data_aug_args to see details for those paramters
data_aug_args = dict(CoarseDropout_range=(0.0, 0.05),
                     CoarseDropout_size_percent=(0.02, 0.25),
                     Affine_translate_percent=(-0.2, 0.2),
                     Affine_scale=(0.5, 1.5),
                     Affine_shear=(-20, 20),
                     Affine_rotate=(-45, 45),
                     Flip_percent=0.5,
                     GaussianBlur_sigma=(0.0, 3.0),
                     CropAndPad_percent=(-0.25, 0.25),
                     Multiply=(0.5, 1.5),
                     LinearContrast=(0.4, 1.6),
                     AdditiveGaussianNoise_scale=0.2*255)
# get augmnetation function
# complicated=True: augmentation for network 2, false: augmentation for network 1
aug = data_augmentation(complicated=False, **data_aug_args)

# get augmented data generator
train_generator_aug, val_generator_aug = load_data(data_path, img_width=64, img_height=64, 
                                                   batch_size=128, augmentation=aug, 
                                                   seed=101)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!


## 4. Model 2 Implementation

In [19]:
import numpy as np
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [ ]:
model2 = init_model_2((64,64,3))

In [14]:
model2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 64, 64, 32)   896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 64, 64, 32)   128         conv2d_5[0][0]                   
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 64, 64, 32)   0           batch_normalization_6[0][0]      
______________________________________________________________________________________________

The orginal paper used Cyclic Learning Rate, we apply the same CyclicLR function using the same code from https://github.com/ZohebAbai/Tiny-ImageNet-Challenge/blob/master/TinyImageNet_Network_2.ipynb

In [15]:
!pip install git+https://github.com/keras-team/keras-contrib.git
from keras_contrib.callbacks import CyclicLR

  Cloning https://github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-4j85hltb
  Running command git clone -q https://github.com/keras-team/keras-contrib.git /tmp/pip-req-build-4j85hltb
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=d543814d6952b9596278ae40ad848fa1fbd4ab2938cfa7a1adac1f2ac09012b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-tui8j_3v/wheels/eb/42/ea/ef324c6958836b1a3d4c0659214b0bae1c0a9bf151254dc93f
Successfully built keras-contrib


In [16]:
opt1=RMSprop(lr= 1e-4, epsilon=1e-08)
opt2= Adam(lr= 1e-4, epsilon=1e-08)
opt3= SGD(momentum=0.9)


model2.compile(loss='categorical_crossentropy',
              optimizer=opt2,
              metrics=['accuracy'])

# After disconnecting and saving the model, run the new model from here after skipping the above part.

In [18]:
from keras.models import load_model
new_model2 = load_model("/content/drive/MyDrive/Colab Notebooks/4040/checkpoints/model2_12-19_22-11.07.h5")

In [19]:
# get augmnetation function
# complicated=True: augmentation for network 2, false: augmentation for network 1
aug = data_augmentation(complicated=True, **data_aug_args)
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2')
train_generator_64_2, val_generator_64_2 = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=aug, seed=None)
new_model2, history2 = training(new_model2, 'new_model2', train_generator_64_2, val_generator_64_2, 
                           clr, checkpoint_filepath, epochs=25)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!
Epoch 1/25
200/200 [==============================] - 427s 2s/step - loss: 4.8323 - accuracy: 0.1148 - val_loss: 4.8791 - val_accuracy: 0.1096
Epoch 2/25
200/200 [==============================] - 410s 2s/step - loss: 4.7780 - accuracy: 0.1199 - val_loss: 4.8131 - val_accuracy: 0.1173
Epoch 3/25
200/200 [==============================] - 408s 2s/step - loss: 4.7319 - accuracy: 0.1229 - val_loss: 4.8201 - val_accuracy: 0.1113
Epoch 4/25
200/200 [==============================] - 404s 2s/step - loss: 4.6740 - accuracy: 0.1259 - val_loss: 4.8851 - val_accuracy: 0.1101
Epoch 5/25
200/200 [==============================] - 400s 2s/step - loss: 4.6501 - accuracy: 0.1300 - val_loss: 4.7994 - val_accuracy: 0.1186
Epoch 6/25
200/200 [==============================] - 402s 2s/step - loss: 4.6055 -

In [20]:
save_model(new_model2, 'new_model2', model_path+'/models')

new_model2_12-20_02-05.h5 saved at /content/drive/MyDrive/Colab Notebooks/4040/savedmodel/models!


In [23]:
model2550, history3 = training(new_model2, 'model2550', train_generator_64_2, val_generator_64_2, 
                           clr, checkpoint_filepath, epochs=25)

Epoch 1/25
200/200 [==============================] - 408s 2s/step - loss: 4.0386 - accuracy: 0.2176 - val_loss: 4.4027 - val_accuracy: 0.1669
Epoch 2/25
200/200 [==============================] - 392s 2s/step - loss: 4.0262 - accuracy: 0.2191 - val_loss: 4.3011 - val_accuracy: 0.1777
Epoch 3/25
200/200 [==============================] - 393s 2s/step - loss: 3.9722 - accuracy: 0.2270 - val_loss: 4.3521 - val_accuracy: 0.1773
Epoch 4/25
200/200 [==============================] - 388s 2s/step - loss: 3.9237 - accuracy: 0.2329 - val_loss: 4.2757 - val_accuracy: 0.1857
Epoch 5/25
200/200 [==============================] - 389s 2s/step - loss: 3.8752 - accuracy: 0.2446 - val_loss: 4.1495 - val_accuracy: 0.2008
Epoch 6/25
200/200 [==============================] - 390s 2s/step - loss: 3.8420 - accuracy: 0.2497 - val_loss: 4.1867 - val_accuracy: 0.2024
Epoch 7/25
200/200 [==============================] - 390s 2s/step - loss: 3.8034 - accuracy: 0.2548 - val_loss: 4.3079 - val_accuracy: 0.1894

In [25]:
save_model(model2550, 'model2550', model_path+'/models')


model2550_12-20_05-29.h5 saved at /content/drive/MyDrive/Colab Notebooks/4040/savedmodel/models!


In [ ]:
model50100, history4 = training(model2550, 'model50100', train_generator_64_2, val_generator_64_2, 
                           clr, checkpoint_filepath, epochs=50)

Epoch 1/50
200/200 [==============================] - 408s 2s/step - loss: 3.1033 - accuracy: 0.3681 - val_loss: 3.4591 - val_accuracy: 0.3070
Epoch 2/50
200/200 [==============================] - 402s 2s/step - loss: 3.1107 - accuracy: 0.3659 - val_loss: 3.3411 - val_accuracy: 0.3257
Epoch 3/50
200/200 [==============================] - 398s 2s/step - loss: 3.1141 - accuracy: 0.3623 - val_loss: 3.4054 - val_accuracy: 0.3126
Epoch 4/50
200/200 [==============================] - 395s 2s/step - loss: 3.1153 - accuracy: 0.3665 - val_loss: 3.4959 - val_accuracy: 0.3022
Epoch 5/50
200/200 [==============================] - 397s 2s/step - loss: 3.1256 - accuracy: 0.3602 - val_loss: 3.4910 - val_accuracy: 0.3054
Epoch 6/50
200/200 [==============================] - 396s 2s/step - loss: 3.1183 - accuracy: 0.3650 - val_loss: 3.5017 - val_accuracy: 0.3033
Epoch 7/50
200/200 [==============================] - 396s 2s/step - loss: 3.1170 - accuracy: 0.3636 - val_loss: 3.4973 - val_accuracy: 0.3009

# After disconnecting and saving the model, run the new model from here after skipping the above part.

In [25]:
from keras.models import load_model
model5083 = load_model("/content/drive/MyDrive/Colab Notebooks/4040/checkpoints/model50100_12-20_05-33.33.h5")


In [31]:


aug = data_augmentation(complicated=True, **data_aug_args)
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2')
train_generator_64_2, val_generator_64_2 = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=aug, seed=None)
num_train=80000
batch_size=256
model83103, history5 = training(model5083, 
                                'model83103', 
                                train_generator_64_2, 
                                val_generator_64_2, 
                                clr, 
                                checkpoint_filepath, 
                                epochs=20, 
                                steps_per_epoch=num_train // batch_size)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!
Epoch 1/20
312/312 [==============================] - 520s 2s/step - loss: 2.8787 - accuracy: 0.4221 - val_loss: 3.2146 - val_accuracy: 0.3648
Epoch 2/20
312/312 [==============================] - 521s 2s/step - loss: 2.8075 - accuracy: 0.4403 - val_loss: 3.2242 - val_accuracy: 0.3632
Epoch 3/20
312/312 [==============================] - 520s 2s/step - loss: 2.8165 - accuracy: 0.4336 - val_loss: 3.3352 - val_accuracy: 0.3480
Epoch 4/20
312/312 [==============================] - 519s 2s/step - loss: 2.8569 - accuracy: 0.4265 - val_loss: 3.6036 - val_accuracy: 0.3007
Epoch 5/20
312/312 [==============================] - 515s 2s/step - loss: 2.9444 - accuracy: 0.4066 - val_loss: 3.4252 - val_accuracy: 0.3257
Epoch 6/20
312/312 [==============================] - 529s 2s/step - loss: 2.9588 -

In [33]:
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2')
train_generator_64_2, val_generator_64_2 = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=aug, seed=None)
num_train=80000
batch_size=128
model103123, history6= training(model83103, 
                                'model103123', 
                                train_generator_64_2, 
                                val_generator_64_2, 
                                clr, 
                                checkpoint_filepath, 
                                epochs=20, 
                                steps_per_epoch=num_train // batch_size)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!
Epoch 1/20
625/625 [==============================] - 925s 1s/step - loss: 2.9957 - accuracy: 0.4379 - val_loss: 3.2187 - val_accuracy: 0.3982
Epoch 2/20
625/625 [==============================] - 938s 2s/step - loss: 2.8773 - accuracy: 0.4604 - val_loss: 3.1481 - val_accuracy: 0.4095
Epoch 3/20
625/625 [==============================] - 951s 2s/step - loss: 2.8131 - accuracy: 0.4729 - val_loss: 3.1080 - val_accuracy: 0.4173
Epoch 4/20
625/625 [==============================] - 941s 2s/step - loss: 2.7676 - accuracy: 0.4775 - val_loss: 3.0765 - val_accuracy: 0.4202
Epoch 5/20
625/625 [==============================] - 942s 2s/step - loss: 2.7244 - accuracy: 0.4833 - val_loss: 3.0576 - val_accuracy: 0.4195
Epoch 6/20
625/625 [==============================] - 943s 2s/step - loss: 2.6916 -

KeyboardInterrupt: ignored

# After disconnecting and saving the model, run the new model from here after skipping the above part.

In [16]:

from keras.models import load_model
model114 = load_model("/content/drive/MyDrive/Colab Notebooks/4040/checkpoints/model114_12-21_00-37.01.h5")


In [17]:
opt1=RMSprop(lr= 1e-4, epsilon=1e-08)
opt2= Adam(lr= 1e-4, epsilon=1e-08)
opt3= SGD(momentum=0.9)
model114.compile(loss='categorical_crossentropy',
              optimizer=opt2,
              metrics=['accuracy'])

In [19]:
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2')
train_generator_64_2, val_generator_64_2 = load_data(data_path, img_width=64, img_height=64, 
                                           batch_size=128, augmentation=aug, seed=None)
num_train=80000
batch_size=128
model115, history7= training(model114, 
                                'model115', 
                                train_generator_64_2, 
                                val_generator_64_2, 
                                clr, 
                                checkpoint_filepath, 
                                epochs=12, 
                                steps_per_epoch=num_train // batch_size)

Begin loading...
Found 80000 images belonging to 200 classes.
Found 20000 images belonging to 200 classes.
Training data shape: (80000, 64, 64, 3)
Validation data shape: (20000, 64, 64, 3)
End loading!
Epoch 1/12
625/625 [==============================] - 542s 866ms/step - loss: 2.9124 - accuracy: 0.4384 - val_loss: 3.1704 - val_accuracy: 0.3922
Epoch 2/12
625/625 [==============================] - 540s 863ms/step - loss: 2.7345 - accuracy: 0.4707 - val_loss: 3.0967 - val_accuracy: 0.4050
Epoch 3/12
625/625 [==============================] - 540s 864ms/step - loss: 2.6657 - accuracy: 0.4850 - val_loss: 3.0414 - val_accuracy: 0.4132
Epoch 4/12
625/625 [==============================] - 540s 863ms/step - loss: 2.6114 - accuracy: 0.4955 - val_loss: 2.9901 - val_accuracy: 0.4249
Epoch 5/12
625/625 [==============================] - 539s 862ms/step - loss: 2.5641 - accuracy: 0.5032 - val_loss: 2.9989 - val_accuracy: 0.4180
Epoch 6/12
625/625 [==============================] - 540s 863ms/ste